In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special

In [ ]:
# implement fourier functions

from typing import Any


def discrete_convolute(signal1: list[Any], signal2: list[Any]):
    """
    Convolute the two signals of the same length
    """

    # flipping signal2 and slide it across signal1, summing the results
    s1 = np.array(signal1, dtype=np.float64)
    s2 = np.array(signal2, dtype=np.float64)

    if s1.shape[0] != s2.shape[0]:
        raise RuntimeError("cannot convolute two signals of different lengths, please pad them first")

    N = s1.shape[0]
    result = []

    # using the head of the flipped s2
    # fuck the indexing here
    for head in np.arange(2*N-1):
        start = max(0, head-(N-1))
        stop  = min(N, head+1)

        summed = np.dot(s1[start:stop], s2[head-stop+1:head-start+1][::-1])
        result.append(summed)

    return result


def fft_convolute(signal1, signal2):
    """
    convolutes the two signals using the convolution theorem

    the signals must be padded using N-1 zeros at the end (where N = len(signal))
    """
    s1 = np.array(signal1, dtype=np.float64)
    s2 = np.array(signal2, dtype=np.float64)

    S1 = np.fft.fft(s1)
    S2 = np.fft.fft(s2)
    return np.fft.ifft(S1 * S2)


# the fft and ifft are implemented in numpy

# integration algorithms
def simple_integral(f, a, b, dx = 0.1):
    if a > b:
        return -sum([
            (f(x) + f(x+dx)) * dx / 2 for x in np.arange(b, a, dx)
        ])

    return sum([
        (f(x) + f(x+dx)) * dx / 2 for x in np.arange(a, b, dx)
    ])


In [ ]:
def testconvolute():
    s1 = [1, 1, 1, 1, 0, 0, 0]
    s2 = [1, 2, 2, 1, 0, 0, 0]

    print(discrete_convolute(s1, s2))
    print(fft_convolute(s1, s2).real)
    print(np.convolve(s1, s2))

testconvolute()

In [ ]:
from dataclasses import dataclass
from typing import Callable
import math

# probability distribution functions


@dataclass
class Domain:
    lower: float
    upper: float
    step: float = 1

    def as_range(self):
        return np.arange(self.lower, self.upper, self.step)

    def length(self):
        return self.as_range().size


@dataclass
class Distribution:
    values: list  # can also be a numpy array, list like
    domain: Domain
    mode: str = 'continuous'

    @staticmethod
    def create(pdf: Callable[[float], float], domain: Domain, mode: str = 'continuous'):

        values = pdf(domain.as_range())

        return Distribution(values=values, domain=domain, mode=mode)


    def plot(self, ax: plt.Axes):
        ax.plot(self.domain.as_range(), self.values)
        ax.set_xlabel("x")
        ax.set_ylabel("f(x)")


    def scatter(self, ax: plt.Axes):
        ax.scatter(self.domain.as_range(), self.values)
        ax.set_xlabel("x")
        ax.set_ylabel("f(x)")


    def __add__(self, other):
        return addDistributions(self, other, self.mode)


    def __sub__(self, other):
        return subDistributions(self, other, self.mode)


    def __neg__(self):
        return negDistribution(self)



def addDistributions(d1: Distribution, d2: Distribution, mode: str = 'continuous'):
    # assuming the same domain
    if d1.domain != d2.domain:
        raise RuntimeError("cannot add distributions of different domain, please pad their values first")


    # convolute the values, use mode='same' to strip boundary values
    # only strip the ende
    if mode == 'continuous':
        new_values = np.convolve(d1.values, d2.values, 'same')
    elif mode == 'discrete':
        new_values = np.convolve(d1.values, d2.values)[:d1.domain.length()]
    
    new_values *= d1.domain.step

    return Distribution(values=new_values, domain=d1.domain, mode=mode)


def negDistribution(d: Distribution):
    if np.sign(d.domain.upper) == np.sign(d.domain.lower):
        new_values = np.zeros(d.domain.length())

    # if lower is the limit
    elif abs(d.domain.upper + d.domain.lower) < d.domain.step:
        new_values = d.values[::-1]
    else:
        raise RuntimeError("cannot negate distributions that has unsymmetrical domain, numeric errors will occur")

    
    return Distribution(values=new_values, domain=d.domain, mode=d.mode)


def subDistributions(d1: Distribution, d2: Distribution, mode: str = 'continuous'):
    return addDistributions(d1, negDistribution(d2), mode=mode)


def areaDistribution(d: Distribution, lower: float, upper: float):
    # find lower index
    lowerIndex = round((lower - d.domain.lower) / d.domain.step)
    upperIndex = d.domain.length() - round((d.domain.upper - upper) / d.domain.step)

    return np.sum(
        d.values[lowerIndex:upperIndex]
    ) * d.domain.step



def makeGaussian(mean, std):
    def pdf(x):
        return np.exp(-0.5 * ((x - mean) / std) ** 2) / (std * np.sqrt(2 * np.pi))

    return pdf

def makeBernoulli(p):
    def pdf(x):
        # res is the array holding the vectorized output
        res = np.zeros(x.size)

        # all conditions
        rounded = np.rint(x)

        res[rounded == 0.0] = 1-p
        res[rounded == 1.0] = p
        return res

    return pdf


def choose(n, k):
    return special.gamma(n+1) / special.gamma(k+1) / special.gamma(n-k+1)


def makeBinominal(n, p):
    def pdf(x):
        # set zero to all x > n
        output = choose(n, x) * p ** x * (1-p) ** (n-x)
        output[np.round(x) > n] = 0
        return output
        
    return pdf


def makeUniform(lower, upper):
    scale = 1/(upper-lower)
    def pdf(x):
        zeros = np.zeros(x.size)
        zeros[(lower < x) & (x < upper)] = scale
        return zeros

    return pdf



In [ ]:

# testing



def test_gaussian():
    domain = Domain(lower=-10, upper=10, step=0.1)
    X = Distribution.create(makeGaussian(-2, 1), domain)
    Y = Distribution.create(makeGaussian(2, 1), domain)
    Z = X + Y
    H = X - Y


    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(7, 9), dpi=120)
    X.plot(ax1)
    Y.plot(ax2)
    Z.plot(ax3)
    H.plot(ax4)
    plt.tight_layout()

def test_bino():
    domain = Domain(lower=0, upper=10, step=0.1)
    X = Distribution.create(makeBernoulli(0.7), domain)
    Ys = list(map(lambda n: Distribution.create(makeBinominal(n, 0.7), domain), range(1,10)))

    fig, ax = plt.subplots(1, figsize=(7, 4), dpi=120)
    for Y in Ys:
        Y.plot(ax)

    ax.set_ylim((0, 1))
    plt.tight_layout()


def test_bern():
    domain = Domain(lower=0, upper=10, step=1)
    X = Distribution.create(makeBinominal(1, 0.7), domain, 'discrete')
    Y = X + X + X + X + X

    fig, (ax1, ax2) = plt.subplots(2, figsize=(7, 7), dpi=120)
    X.plot(ax1)
    Y.plot(ax2)

    plt.tight_layout()


test_gaussian()

In [ ]:
#  compute geometric probability

domain = Domain(-120, 120, 0.01)
lydia = Distribution.create(makeUniform(0, 60), domain)
# lydia = Distribution.create(makeGaussian(30, 10), domain)
rania = Distribution.create(makeUniform(0, 60), domain)
# rania = Distribution.create(makeGaussian(30, 10), domain)

D = lydia - rania
print(areaDistribution(D, -15, 15))

# print(areaDistribution(lydia, 0, 60))

fig, ax = plt.subplots(1, dpi=120)
D.plot(ax)
plt.tight_layout()